In [ ]:
using JLD2
using FileIO
using DataFrames
using Dates

# נגדיר את הנתיב הראשי שבו נמצאות כל התוצאות
# שים לב: זה הנתיב החיצוני (הכונן הקשיח) שציינת
base_data_path = "/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep"